In [1]:
# YOU HAVE TO SOURCE THE lattice/lattice/bin/activate virtual env for this, not conda!

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tempfile

import sys
!{sys.executable} -m pip install pyarrow

import pandas as pd
import six
import tensorflow as tf
import tensorflow_lattice as tfl
import timeit
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import sys
sys.path.append("./lattice/lib/python3.6/site-packages")
import pyarrow
print(pyarrow.__path__)

You should consider upgrading via the 'pip install --upgrade pip' command.
['/local/ssd/home/mattfel/slac/timetool-ML-models/lattice/lattice/lib/python3.6/site-packages/pyarrow']


In [ ]:
# Import helpers
import import_ipynb
from common import buildDataset
from common import scaleVolume
from common import extractLatticeWeights
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractXGWeights

In [ ]:
dataset, columns = buildDataset()
dataset = filterBad(dataset)
dataset = scaleVolume(dataset, 4096)
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

In [ ]:
create_quantiles=False
input_fn = tf.estimator.inputs.pandas_input_fn(
      x=train_dataset,
      y=train_labels,
      batch_size=10000,
      shuffle=False,
      num_epochs=100,
      num_threads=1)

test_input_fn = tf.estimator.inputs.pandas_input_fn(
      x=test_dataset,
      y=test_labels,
      batch_size=10000,
      shuffle=False,
      num_epochs=1,
      num_threads=1)

feature_columns = [
    tf.feature_column.numeric_column("row"),
    tf.feature_column.numeric_column("rising_idx"),
    tf.feature_column.numeric_column("falling_idx"),
    tf.feature_column.numeric_column("first_val"),
    tf.feature_column.numeric_column("last_val"),
                  ]
if (create_quantiles): tfl.save_quantiles_for_keypoints(
  input_fn=input_fn,
  save_dir='./quantiles',
  feature_columns=feature_columns,
  num_steps=None)

In [63]:
def buildLattice(NUM_KEYPOINTS, LATTICE_SIZE, interp_type):
    feature_names = [fc.name for fc in feature_columns]
    hparams = tfl.CalibratedLatticeHParams(
      feature_names=feature_names,
      learning_rate=20,
      lattice_l2_laplacian_reg=5.0e-4, 
      lattice_l2_torsion_reg=1.0e-4,
      interpolation_type=interp_type,
      num_keypoints=NUM_KEYPOINTS, 
      # feature__outerIters__monotonicity=1,
      # feature__outerIters__lattice_size=3,
      # monotonicity=frozenset({'outerIters': 1}.items),
      lattice_size=LATTICE_SIZE, 
      lattice_rank=len(feature_columns),
      num_lattices=1) 

    # Specific feature parameters.
#     _pprint_hparams(hparams)

    return tfl.calibrated_lattice_regressor(
            feature_columns=feature_columns,
            model_dir='./output',
            quantiles_dir='./quantiles',
            keypoints_initializers_fn=None,
            optimizer=tf.train.AdagradOptimizer,
#             optimizer=tf.train.AdamOptimizer,
            hparams=hparams
        )

In [64]:
def trainLattice(model):
    model.train(input_fn=input_fn)
    return model
    

In [65]:
# show_perf = True
# train_model = True


# Hypercube_8_2 = buildLattice(2,8,'hypercube')
# if (train_model): Hypercube_8_2 = trainLattice(Hypercube_8_2) #train
# else: Hypercube_8_2.load_weights('./Hypercube_8_2') #load
# Hypercube_8_2.save_weights('./Hypercube_8_2')
# preds = Hypercube_8_2.predict(train_dataset)
# if (show_perf): evaluatePerf(test_labels, preds)
# print('--------------------------------')
# extractLatticeWeights(Hypercube_8_2, 'Hypercube_8_2')
# print('--------------------------------')


In [ ]:
show_perf = True
train_model = True


Simplex_8_2 = buildLattice(2,8,'simplex')
if (train_model): Simplex_8_2 = trainLattice(Simplex_8_2) #train
else: Simplex_8_2.load_weights('./Simplex_8_2') #load
preds = Simplex_8_2.predict(train_dataset)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractLatticeWeights(Simplex_8_2, 'Simplex_8_2')
print('--------------------------------')


INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9e74585278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9e74585278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 5350 into ./output/model.ckpt.


INFO:tensorflow:global_step/sec: 0.415194


INFO:tensorflow:global_step/sec: 0.415194


INFO:tensorflow:loss = 768978240.0, step = 5401 (240.850 sec)


INFO:tensorflow:loss = 768978240.0, step = 5401 (240.850 sec)


In [ ]:

preds = Simplex_8_2.predict(test_input_fn)
results   = list(preds);
print(results)
tf.logging.info(results);
if (show_perf): evaluatePerf(test_labels, preds)

In [ ]:
print('hi')